In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv('final_crawled_api_data.csv')
df.head()

,title,link,mountain,upload_to,운해 여부,위치,전날 기온,전날 강수량,전날 습도,전날 기압,일출 기온,일출 풍속,일출 습도,일출 기압,일출 전운량,일교차
0,오늘 일출과 운해,https://cafe.naver.com/windstopper/ArticleRead...,[‘심항산’],20230508.0,1.0,충주,5.8,0.0,84.0,1001.4,3.2,0.0,93.0,1003.0,0.0,-2.6
1,"노동절 새벽, 월출산의 운해일출",https://cafe.naver.com/windstopper/ArticleRead...,['월출산'],20230501.0,1.0,강진,8.9,0.0,91.0,1012.2,9.3,0.8,92.0,1012.4,7.0,0.4
2,백운대 일출산행(feat. 운해사냥),https://cafe.naver.com/windstopper/ArticleRead...,[‘북한산'],20230501.0,1.0,서울,12.1,0.0,87.0,1001.3,10.1,2.4,91.0,1002.7,7.0,-2.0
3,백운대 운해사냥성공(thanks to또민아빠님),https://cafe.naver.com/windstopper/ArticleRead...,[‘북한산'],20230430.0,1.0,서울,10.0,0.0,83.0,998.3,7.9,2.5,87.0,998.9,0.0,-2.1
4,지금 이 순간(역대급 제비봉 운해),https://cafe.naver.com/windstopper/ArticleRead...,[‘제비봉’],20230427.0,1.0,제천,1.7,0.0,100.0,987.3,-0.3,0.0,100.0,989.1,10.0,-2.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 807 entries, 0 to 806
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      806 non-null    object 
 1   link       806 non-null    object 
 2   mountain   805 non-null    object 
 3   upload_to  806 non-null    float64
 4   운해 여부      806 non-null    float64
 5   위치         806 non-null    object 
 6   전날 기온      806 non-null    float64
 7   전날 강수량     806 non-null    float64
 8   전날 습도      805 non-null    float64
 9   전날 기압      806 non-null    float64
 10  일출 기온      806 non-null    float64
 11  일출 풍속      805 non-null    float64
 12  일출 습도      805 non-null    float64
 13  일출 기압      806 non-null    float64
 14  일출 전운량     806 non-null    float64
 15  일교차        806 non-null    float64
dtypes: float64(12), object(4)
memory usage: 101.0+ KB


In [8]:
df=df.drop(['title', 'link', 'upload_to', 'mountain', '위치'],axis=1)
df

,운해 여부,전날 기온,전날 강수량,전날 습도,전날 기압,일출 기온,일출 풍속,일출 습도,일출 기압,일출 전운량,일교차
0,1.0,5.8,0.0,84.0,1001.4,3.2,0.0,93.0,1003.0,0.0,-2.6
1,1.0,8.9,0.0,91.0,1012.2,9.3,0.8,92.0,1012.4,7.0,0.4
2,1.0,12.1,0.0,87.0,1001.3,10.1,2.4,91.0,1002.7,7.0,-2.0
3,1.0,10.0,0.0,83.0,998.3,7.9,2.5,87.0,998.9,0.0,-2.1
4,1.0,1.7,0.0,100.0,987.3,-0.3,0.0,100.0,989.1,10.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...
802,0.0,-10.8,0.0,89.0,941.9,-13.1,0.3,87.0,941.8,0.0,-2.3
803,0.0,-4.5,0.0,61.0,1008.7,-7.0,0.6,72.0,1008.7,0.0,-2.5
804,0.0,8.5,0.0,76.0,1015.7,7.7,0.3,85.0,1015.7,0.0,-0.8
805,0.0,-5.0,0.0,73.0,1009.7,-5.8,0.0,83.0,1009.7,0.0,-0.8


In [11]:
df.columns

Index(['운해 여부', '전날 기온', '전날 강수량', '전날 습도', '전날 기압', '일출 기온', '일출 풍속', '일출 습도',
       '일출 기압', '일출 전운량', '일교차'],
      dtype='object')

In [10]:
df=df.dropna()

In [12]:
from sklearn.model_selection import train_test_split

X = df[['전날 기온', '전날 강수량', '전날 습도', '전날 기압', '일출 기온', '일출 풍속', '일출 습도',
       '일출 기압', '일출 전운량', '일교차']]
y = df[['운해 여부']]

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(643, 10) (161, 10) (643, 1) (161, 1)


In [13]:
from sklearn.linear_model import LogisticRegression      ## 로지스틱 회귀분석
from sklearn.naive_bayes import GaussianNB               ## 나이브 베이즈

In [14]:
clf = LogisticRegression(solver='lbfgs').fit(X_train,y_train)

gnb = GaussianNB()
gnb.fit(X_train,y_train)

print(clf.score(X_test,y_test))
print(gnb.score(X_test,y_test))

0.6894409937888198
0.6335403726708074


C:\Users\leeji\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\leeji\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\leeji\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example us

In [15]:
heatmap_data=df

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os 
if os.name =='nt':
    font_family ="Malgun Gothic"
else:
    font_family ="AppleGothic"
    
sns.set(font=font_family, rc={"axes.unicode_minus": False})

In [ ]:
colormap = plt.cm.PuBu
plt.figure(figsize=(10, 8))
plt.title("Person Correlation of Features", y = 1.05, size = 30)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths = 0.1, vmax = 1.0,
           square = True, cmap = colormap, linecolor = "white", annot = True, annot_kws = {"size" : 16})


<Axes: title={'center': 'Person Correlation of Features'}>